In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Kuunda Mteja

Katika mfano huu, tutatumia [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) kwa ufikiaji wa LLM.

`model` imefafanuliwa kama `gpt-4o-mini`. Jaribu kubadilisha modeli na modeli nyingine inayopatikana kwenye soko la GitHub Models ili kuona matokeo tofauti.

Kama jaribio la haraka, tutakimbiza tu ombi rahisi - `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Kufafanua Kazi

Katika mfano huu, tutampa wakala uwezo wa kutumia zana ambayo ni kazi yenye orodha ya maeneo ya likizo yanayopatikana na upatikanaji wake.

Unaweza kufikiria kuwa hii itakuwa hali ambapo wakala wa kusafiri anaweza kuwa na ufikiaji wa hifadhidata ya safari kwa mfano.

Unapopitia mfano huu, jisikie huru kujaribu kufafanua kazi mpya na zana ambazo wakala anaweza kutumia.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Kuweka Kifaa cha Kazi cha Kazi  
Ili wakala aweze kutumia `vacation_destinations` kama `FunctionTool`, tunahitaji kukifafanua kama kifaa cha kazi.  

Pia tutatoa maelezo ya kifaa hicho ambayo yatamsaidia wakala kutambua kifaa hicho kinatumika kwa nini kuhusiana na kazi ambayo mtumiaji ameomba.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Kuweka wakala

Sasa tunaweza kuunda wakala katika msimbo ulio hapa chini. Tunafafanua `system_message` ili kumpa wakala maelekezo ya jinsi ya kuwasaidia watumiaji kupata maeneo ya likizo. 

Pia tunawasha kipengele cha `reflect_on_tool_use` kwa kuweka thamani yake kuwa kweli. Hii inaruhusu LLM kuchukua majibu ya matumizi ya zana na kutuma majibu kwa kutumia lugha ya kawaida. 

Unaweza kuweka thamani ya kipengele kuwa si kweli ili kuona tofauti.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Kuendesha Wakala

Sasa tunaweza kuendesha wakala na ujumbe wa awali wa mtumiaji akiuliza kuhusu safari ya kwenda Tokyo.

Unaweza kubadilisha mji huu wa marudio ili kuona jinsi wakala anavyojibu kuhusu upatikanaji wa mji huo.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya kutafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
